# GPT Finetuning KorQuAD



## 데이터셋 준비

In [ ]:
!wget https://raw.githubusercontent.com/korquad/korquad.github.io/refs/heads/master/dataset/KorQuAD_v1.0_dev.json

--2025-06-27 01:26:07--  https://raw.githubusercontent.com/korquad/korquad.github.io/refs/heads/master/dataset/KorQuAD_v1.0_dev.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [text/plain]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[===================>]   3.70M  --.-KB/s    in 0.09s   

2025-06-27 01:26:07 (43.2 MB/s) - ‘KorQuAD_v1.0_dev.json’ saved [3881058/3881058]



## 데이터준비

In [ ]:
import json

with open('KorQuAD_v1.0_dev.json', 'r', encoding='utf-8') as f:
  dev_data = json.load(f)

dev_data['data'][0]

{'paragraphs': [{'qas': [{'answers': [{'text': '1989년 2월 15일',
       'answer_start': 0}],
     'id': '6548850-0-0',
     'question': '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?'},
    {'answers': [{'text': '임수경', 'answer_start': 125}],
     'id': '6548850-0-1',
     'question': '1989년 6월 30일 평양축전에 대표로 파견 된 인물은?'},
    {'answers': [{'text': '1989년', 'answer_start': 0}],
     'id': '6548853-0-0',
     'question': '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?'},
    {'answers': [{'text': '학생회관 건물 계단', 'answer_start': 365}],
     'id': '6548853-0-1',
     'question': '임종석을 검거한 장소는 경희대 내 어디인가?'},
    {'answers': [{'text': '서울지방경찰청 공안분실', 'answer_start': 457}],
     'id': '6548853-0-2',
     'question': '임종석이 조사를 받은 뒤 인계된 곳은 어딘가?'},
    {'answers': [{'text': '임종석', 'answer_start': 87}],
     'id': '6332405-0-0',
     'question': '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?'},
    {'answers': [{'text': '여의도 농민 폭력 시위', 'answer_start': 13}],
     'id': '6332405-0-1',
     'question': '임종석이 1989년 2월 

## 데이터 정제
데이터가 너무 많아 20건 정도만 만들 예정

In [ ]:
items = [item for topic in dev_data['data'] for item in topic['paragraphs']]
items[0]

{'qas': [{'answers': [{'text': '1989년 2월 15일', 'answer_start': 0}],
   'id': '6548850-0-0',
   'question': '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?'},
  {'answers': [{'text': '임수경', 'answer_start': 125}],
   'id': '6548850-0-1',
   'question': '1989년 6월 30일 평양축전에 대표로 파견 된 인물은?'},
  {'answers': [{'text': '1989년', 'answer_start': 0}],
   'id': '6548853-0-0',
   'question': '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?'},
  {'answers': [{'text': '학생회관 건물 계단', 'answer_start': 365}],
   'id': '6548853-0-1',
   'question': '임종석을 검거한 장소는 경희대 내 어디인가?'},
  {'answers': [{'text': '서울지방경찰청 공안분실', 'answer_start': 457}],
   'id': '6548853-0-2',
   'question': '임종석이 조사를 받은 뒤 인계된 곳은 어딘가?'},
  {'answers': [{'text': '임종석', 'answer_start': 87}],
   'id': '6332405-0-0',
   'question': '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?'},
  {'answers': [{'text': '여의도 농민 폭력 시위', 'answer_start': 13}],
   'id': '6332405-0-1',
   'question': '임종석이 1989년 2월 15일에 지명수배 받은 혐의는 어떤 시위를 주도했다는 것인가?'}],
 'context': '1989년 2월 15

In [ ]:
qa_dict = {}

for item in items:
    for entry in item['qas']:
        question = entry['question']
        answer = [ans['text'] for ans in entry['answers']][0]
        qa_dict[question] = answer

qa_dict

{'임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?': '1989년 2월 15일',
 '1989년 6월 30일 평양축전에 대표로 파견 된 인물은?': '임수경',
 '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?': '1989년',
 '임종석을 검거한 장소는 경희대 내 어디인가?': '학생회관 건물 계단',
 '임종석이 조사를 받은 뒤 인계된 곳은 어딘가?': '서울지방경찰청 공안분실',
 '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?': '임종석',
 '임종석이 1989년 2월 15일에 지명수배 받은 혐의는 어떤 시위를 주도했다는 것인가?': '여의도 농민 폭력 시위',
 '정부의 헌법개정안 준비 과정에 대해서 청와대 비서실이 아니라 국무회의 중심으로 이뤄졌어야 했다고 지적한 원로 헌법학자는?': '허영',
 "'행보가 비서 본연의 역할을 벗어난다', '장관들과 내각이 소외되고 대통령비서실의 권한이 너무 크다'는 의견이 제기된 대표적인 예는?": '10차 개헌안 발표',
 '국무회의의 심의를 거쳐야 한다는 헌법 제 몇 조의 내용인가?': '제89조',
 '법무부 장관을 제쳐놓고 민정수석이 개정안을 설명하는 게 이해가 안 된다고 지적한 경희대 석좌교수 이름은?': '허영',
 '미국 군대 내 두번째로 높은 직위는 무엇인가?': '미국 육군 부참모 총장',
 '로널드 레이건 정부 출범 당시 알렉산더 헤이그는 어떤 직책을 맡았는가?': '초대 국무장관직',
 '알렉산더 헤이그는 어느 대통령의 밑에서 국무장관을 지냈는가?': '로널드 레이건 대통령',
 '로널드 레이건 대통령 밑에서 일한 국무 장관은 누구인가?': '알렉산더 메이그스 헤이그 2세',
 '미국 군대에서 두번째로 높은 직위는?': '미국 육군 부참모 총장',
 '알렉산더 메이그스 헤이그의 생년월일은?': '1924년 12월 2일',
 '알렉산더 헤이그가 로널드 레이건 대통령 밑에서 맡은 직책은 무엇이었나?': '국무장

In [ ]:
len(qa_dict)

5764

In [ ]:
# 20건만
qa_dict = dict(list(qa_dict.items())[:20])
qa_dict

{'임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?': '1989년 2월 15일',
 '1989년 6월 30일 평양축전에 대표로 파견 된 인물은?': '임수경',
 '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?': '1989년',
 '임종석을 검거한 장소는 경희대 내 어디인가?': '학생회관 건물 계단',
 '임종석이 조사를 받은 뒤 인계된 곳은 어딘가?': '서울지방경찰청 공안분실',
 '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?': '임종석',
 '임종석이 1989년 2월 15일에 지명수배 받은 혐의는 어떤 시위를 주도했다는 것인가?': '여의도 농민 폭력 시위',
 '정부의 헌법개정안 준비 과정에 대해서 청와대 비서실이 아니라 국무회의 중심으로 이뤄졌어야 했다고 지적한 원로 헌법학자는?': '허영',
 "'행보가 비서 본연의 역할을 벗어난다', '장관들과 내각이 소외되고 대통령비서실의 권한이 너무 크다'는 의견이 제기된 대표적인 예는?": '10차 개헌안 발표',
 '국무회의의 심의를 거쳐야 한다는 헌법 제 몇 조의 내용인가?': '제89조',
 '법무부 장관을 제쳐놓고 민정수석이 개정안을 설명하는 게 이해가 안 된다고 지적한 경희대 석좌교수 이름은?': '허영',
 '미국 군대 내 두번째로 높은 직위는 무엇인가?': '미국 육군 부참모 총장',
 '로널드 레이건 정부 출범 당시 알렉산더 헤이그는 어떤 직책을 맡았는가?': '초대 국무장관직',
 '알렉산더 헤이그는 어느 대통령의 밑에서 국무장관을 지냈는가?': '로널드 레이건 대통령',
 '로널드 레이건 대통령 밑에서 일한 국무 장관은 누구인가?': '알렉산더 메이그스 헤이그 2세',
 '미국 군대에서 두번째로 높은 직위는?': '미국 육군 부참모 총장',
 '알렉산더 메이그스 헤이그의 생년월일은?': '1924년 12월 2일',
 '알렉산더 헤이그가 로널드 레이건 대통령 밑에서 맡은 직책은 무엇이었나?': '국무장

In [ ]:
results = []

for q, a in qa_dict.items():
    qa = {
        "messages": [
            {"role": "system", "content": ""},
            {"role": "user", "content": q},
            {"role": "assistant", "content": a},
        ]
    }
    results.append(qa)

results

[{'messages': [{'role': 'system', 'content': ''},
   {'role': 'user', 'content': '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?'},
   {'role': 'assistant', 'content': '1989년 2월 15일'}]},
 {'messages': [{'role': 'system', 'content': ''},
   {'role': 'user', 'content': '1989년 6월 30일 평양축전에 대표로 파견 된 인물은?'},
   {'role': 'assistant', 'content': '임수경'}]},
 {'messages': [{'role': 'system', 'content': ''},
   {'role': 'user', 'content': '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?'},
   {'role': 'assistant', 'content': '1989년'}]},
 {'messages': [{'role': 'system', 'content': ''},
   {'role': 'user', 'content': '임종석을 검거한 장소는 경희대 내 어디인가?'},
   {'role': 'assistant', 'content': '학생회관 건물 계단'}]},
 {'messages': [{'role': 'system', 'content': ''},
   {'role': 'user', 'content': '임종석이 조사를 받은 뒤 인계된 곳은 어딘가?'},
   {'role': 'assistant', 'content': '서울지방경찰청 공안분실'}]},
 {'messages': [{'role': 'system', 'content': ''},
   {'role': 'user',
    'content': '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?'},
   {'role': 'assist

## jsonl 파일 생성

In [ ]:
with open('korquad_data.jsonl', 'w', encoding='utf-8') as f:
  for qa in results:
    json_str = json.dumps(qa, ensure_ascii=-False) #ensure_ascii=-False 이렇게 하면 한글로 나옴
    f.write(json_str + '\n') # 한줄에 하나의 json 객체

In [ ]:
from google.colab import files

files.download('korquad_data.jsonl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Fine- tuned 모델 추론

In [1]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [2]:
from openai import OpenAI

model = 'ft:gpt-4.1-2025-04-14:mycookies:korquad20:BmtcD7zO'

client = OpenAI(api_key=OPENAI_API_KEY)

def korquad_finetuned_qa(prompt, temperature=1):
    response = client.chat.completions.create(
        model = model,
        messages = [
            {"role": "system", "content": ""},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature
    )
    return response.choices[0].message.content


In [3]:
korquad_finetuned_qa('법무부 장관을 제쳐놓고 민정수석이 개정안을 설명하는 게 이해가 안 된다고 지적한 경희대 석좌교수 이름은?')

'허영'

In [4]:
korquad_finetuned_qa('알렉산더 헤이그와 1950년 5월 결혼한 상대의 이름은 무엇인가?')

'퍼트리샤 앤 폭스'